In [1]:
import warnings 
with warnings.catch_warnings(): 
    warnings.filterwarnings("ignore",category=FutureWarning)

from time import time

from tensorflow import VERSION as tensorflow_version
from tensorflow.keras import __version__ as keras_version
from tensorflow.keras import Sequential, activations
from tensorflow.keras.layers import Dense, Dropout

from classes import my_tensorflow

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
print(tensorflow_version)
print(keras_version)

1.10.0
2.1.6-tf


In [3]:
TR = my_tensorflow.TensorflowRecognition(
    file_path='C:\\marko\\jupyter\\Statoil-Iceberg-Classifier-Challenge\\data\\processed\\train.json',
    ratio=[0.9, 0.05, 0.05])
X_train, y_train, X_validate, y_validate, X_test, y_test = TR.prepare_datasets()

All data size: (1604, 4)
Size of the whole dataset for training:: (1471, 364)
Split: 1323 73 73
X_train: (1323, 362)
y_train: (1323, 2)
X_validate: (73, 362)
y_validate: (73, 2)
X_test: (75, 362)
y_test: (75, 2)


### The `y` is a 2 dimensional array  with a column for `is_iceberg` and `not_iceberg`. Below is where the two columns are put into one for the purpose of tensorflow training.

In [4]:
y_train = y_train[:,:1]
y_validate = y_validate[:,:1]
y_test = y_test[:,:1]

In [5]:
"""
model = tf.keras.Sequential([
    tf.keras.layers.Dense(dense_1, activation=activation_1),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Dense(dense_2, activation=activation_2)
])
"""

'\nmodel = tf.keras.Sequential([\n    tf.keras.layers.Dense(dense_1, activation=activation_1),\n    tf.keras.layers.Dropout(dropout),\n    tf.keras.layers.Dense(dense_2, activation=activation_2)\n])\n'

# Custom built model
### Sequential
##### https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential
#### Sequential model is most common model. It stacks layers linearly. A fully connected multi-layer perceptron is build below.

### Dense
##### https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense
#### Dense implements `output = activation(dot(input * weights) + bias)`

### Activations
##### https://www.tensorflow.org/api_docs/python/tf/keras/activations
#### Activation functions

### Dropout
##### https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout
#### Dropout randomly assigns value `0` to a fraction of inputs during *training*. This is done to prevent *overfitting*.

In [12]:
dense_1 = 512
dropout = 0.2
dense_2 = 10
use_bias = True
epochs = 300

activation_1 = 'relu'
activation_2 = 'softmax'

optimizer = 'adam'
loss = 'sparse_categorical_crossentropy'
metrics = ['accuracy']

model = Sequential()
# the model takes input in shape (, 362) and outputs array of shape (, 32)
model.add(Dense(units=dense_1, input_shape=(362,), activation=activation_1, use_bias=use_bias))
# dropout is applied
model.add(Dropout(rate=dropout))
model.add(Dense(units=dense_2, input_shape=(dense_1,), activation=activation_2, use_bias=use_bias))

In [13]:
model.compile(optimizer=optimizer,
              loss=loss,
              metrics=metrics)

In [14]:
print("Train:")
t0 = time()
model.fit(X_train, y_train, epochs=epochs, validation_data=(X_validate, y_validate))
t1 = time()
train_time = t1-t0
print("Evaluate:")
val_loss, val_acc = model.evaluate(X_test, y_test)
print(val_loss, val_acc)

TR.write_stats(activation_1, activation_2, optimizer, loss, metrics, dense_1, dropout, dense_2, epochs, use_bias, 
               train_time, val_loss, val_acc)

Train:
Train on 1323 samples, validate on 73 samples
Epoch 1/300
1323/1323 [==============================] - 0s 344us/step - loss: 0.7508 - acc: 0.5911 - val_loss: 0.6287 - val_acc: 0.4795
Epoch 2/300
1323/1323 [==============================] - 0s 122us/step - loss: 0.7215 - acc: 0.5828 - val_loss: 0.5396 - val_acc: 0.7945
Epoch 3/300
1323/1323 [==============================] - 0s 125us/step - loss: 0.6339 - acc: 0.6372 - val_loss: 0.5071 - val_acc: 0.7945
Epoch 4/300
1323/1323 [==============================] - 0s 122us/step - loss: 0.5989 - acc: 0.6515 - val_loss: 0.5120 - val_acc: 0.7671
Epoch 5/300
1323/1323 [==============================] - 0s 121us/step - loss: 0.5882 - acc: 0.6629 - val_loss: 0.4964 - val_acc: 0.7945
Epoch 6/300
1323/1323 [==============================] - 0s 115us/step - loss: 0.5540 - acc: 0.7037 - val_loss: 0.5559 - val_acc: 0.6027
Epoch 7/300
1323/1323 [==============================] - 0s 122us/step - loss: 0.5585 - acc: 0.6954 - val_loss: 0.4899 - val_

1323/1323 [==============================] - 0s 127us/step - loss: 0.3670 - acc: 0.8080 - val_loss: 0.3423 - val_acc: 0.9041
Epoch 61/300
1323/1323 [==============================] - 0s 110us/step - loss: 0.3390 - acc: 0.8360 - val_loss: 0.3056 - val_acc: 0.9041
Epoch 62/300
1323/1323 [==============================] - 0s 109us/step - loss: 0.3362 - acc: 0.8413 - val_loss: 0.2716 - val_acc: 0.8767
Epoch 63/300
1323/1323 [==============================] - 0s 111us/step - loss: 0.3174 - acc: 0.8632 - val_loss: 0.2731 - val_acc: 0.8904
Epoch 64/300
1323/1323 [==============================] - 0s 120us/step - loss: 0.3224 - acc: 0.8337 - val_loss: 0.4446 - val_acc: 0.7397
Epoch 65/300
1323/1323 [==============================] - 0s 111us/step - loss: 0.3338 - acc: 0.8299 - val_loss: 0.3672 - val_acc: 0.8493
Epoch 66/300
1323/1323 [==============================] - 0s 110us/step - loss: 0.3275 - acc: 0.8375 - val_loss: 0.2811 - val_acc: 0.8904
Epoch 67/300
1323/1323 [=======================

1323/1323 [==============================] - 0s 118us/step - loss: 0.2584 - acc: 0.8859 - val_loss: 0.4740 - val_acc: 0.7808
Epoch 120/300
1323/1323 [==============================] - 0s 121us/step - loss: 0.2724 - acc: 0.8730 - val_loss: 0.2550 - val_acc: 0.9041
Epoch 121/300
1323/1323 [==============================] - 0s 111us/step - loss: 0.2691 - acc: 0.8768 - val_loss: 0.2981 - val_acc: 0.8904
Epoch 122/300
1323/1323 [==============================] - 0s 110us/step - loss: 0.2681 - acc: 0.8662 - val_loss: 0.2719 - val_acc: 0.9041
Epoch 123/300
1323/1323 [==============================] - 0s 111us/step - loss: 0.2669 - acc: 0.8670 - val_loss: 0.2726 - val_acc: 0.8630
Epoch 124/300
1323/1323 [==============================] - 0s 110us/step - loss: 0.2625 - acc: 0.8912 - val_loss: 0.3122 - val_acc: 0.8767
Epoch 125/300
1323/1323 [==============================] - 0s 110us/step - loss: 0.2554 - acc: 0.8776 - val_loss: 0.2607 - val_acc: 0.9178
Epoch 126/300
1323/1323 [================

1323/1323 [==============================] - 0s 134us/step - loss: 0.2432 - acc: 0.8813 - val_loss: 0.2790 - val_acc: 0.8904
Epoch 179/300
1323/1323 [==============================] - 0s 124us/step - loss: 0.2508 - acc: 0.8821 - val_loss: 0.3555 - val_acc: 0.8904
Epoch 180/300
1323/1323 [==============================] - 0s 123us/step - loss: 0.2271 - acc: 0.8957 - val_loss: 0.2810 - val_acc: 0.9041
Epoch 181/300
1323/1323 [==============================] - 0s 122us/step - loss: 0.2272 - acc: 0.8980 - val_loss: 0.2698 - val_acc: 0.8904
Epoch 182/300
1323/1323 [==============================] - 0s 123us/step - loss: 0.2264 - acc: 0.8980 - val_loss: 0.2519 - val_acc: 0.9041
Epoch 183/300
1323/1323 [==============================] - 0s 128us/step - loss: 0.2319 - acc: 0.8896 - val_loss: 0.2653 - val_acc: 0.9178
Epoch 184/300
1323/1323 [==============================] - 0s 125us/step - loss: 0.2024 - acc: 0.9085 - val_loss: 0.2697 - val_acc: 0.8767
Epoch 185/300
1323/1323 [================

1323/1323 [==============================] - 0s 122us/step - loss: 0.1927 - acc: 0.9161 - val_loss: 0.3217 - val_acc: 0.8767
Epoch 238/300
1323/1323 [==============================] - 0s 122us/step - loss: 0.2124 - acc: 0.8949 - val_loss: 0.3010 - val_acc: 0.8767
Epoch 239/300
1323/1323 [==============================] - 0s 138us/step - loss: 0.1986 - acc: 0.9138 - val_loss: 0.3158 - val_acc: 0.9041
Epoch 240/300
1323/1323 [==============================] - 0s 141us/step - loss: 0.2175 - acc: 0.9131 - val_loss: 0.2801 - val_acc: 0.8630
Epoch 241/300
1323/1323 [==============================] - 0s 145us/step - loss: 0.2226 - acc: 0.8987 - val_loss: 0.3350 - val_acc: 0.8904
Epoch 242/300
1323/1323 [==============================] - 0s 138us/step - loss: 0.2111 - acc: 0.9048 - val_loss: 0.2176 - val_acc: 0.9178
Epoch 243/300
1323/1323 [==============================] - ETA: 0s - loss: 0.1955 - acc: 0.915 - 0s 123us/step - loss: 0.1962 - acc: 0.9131 - val_loss: 0.3083 - val_acc: 0.8767
Epo

Epoch 296/300
1323/1323 [==============================] - 0s 128us/step - loss: 0.1832 - acc: 0.9146 - val_loss: 0.3481 - val_acc: 0.8904
Epoch 297/300
1323/1323 [==============================] - 0s 119us/step - loss: 0.1567 - acc: 0.9365 - val_loss: 0.3393 - val_acc: 0.8767
Epoch 298/300
1323/1323 [==============================] - 0s 128us/step - loss: 0.1810 - acc: 0.9199 - val_loss: 0.2978 - val_acc: 0.9041
Epoch 299/300
1323/1323 [==============================] - 0s 135us/step - loss: 0.1852 - acc: 0.9138 - val_loss: 0.2981 - val_acc: 0.9041
Epoch 300/300
1323/1323 [==============================] - 0s 122us/step - loss: 0.1809 - acc: 0.9237 - val_loss: 0.2756 - val_acc: 0.8630
Evaluate:
75/75 [==============================] - 0s 80us/step
0.26260109106699625 0.8533333373069764
2019-02-06 15:19:53,relu,softmax,adam,sparse_categorical_crossentropy,['accuracy'],512,0.2,10,300,True,50.62,0.2626,0.8533
